### Problem

What vulnerable entities fall within an individual dams inundation zone (output in a list, showing the different entities for a selected dam)
Count how many vulnerable entities are within a dam's inundation zone (just a number)

In [1]:
import psycopg2
from tabulate import tabulate

In [2]:
db_params = {
    "dbname": "gisdb",
    "user": "admin",
    "password": "admin",
    "host": "localhost",
    "port": 5432,
}

try:
    conn = psycopg2.connect(**db_params)
    print("Connected to the database successfully.")
except Exception as e:
    print(f"Error connecting to the database: {e}")

Connected to the database successfully.


In [3]:
def get_actual_column_names(conn, table, schema="public"):
    """Return the case-sensitive column names for schema.table"""
    with conn.cursor() as cur:
        cur.execute(
            """
            SELECT column_name
            FROM information_schema.columns
            WHERE table_schema = %s AND table_name = %s
            """,
            (schema, table),
        )
        return [r[0] for r in cur.fetchall()]

In [4]:
def list_available_dams(conn, limit=50):
    """Return dams sorted by inundation-polygon area (largest first)."""
    with conn.cursor() as cur:
        dam_cols = get_actual_column_names(conn, "utah_dams")
        name_col = next(c for c in dam_cols if c.upper() == "NAME")
        type_col = next(c for c in dam_cols if c.upper() == "TYPE")
        geom_col = "geom"

        cur.execute(
            f"""
            SELECT "{name_col}", "{type_col}",
                   ST_Area("{geom_col}"::geography) / 1e6 AS area_sq_km
            FROM utah_dams
            ORDER BY area_sq_km DESC
            LIMIT %s
            """,
            (limit,),
        )
        rows = cur.fetchall()

    headers = ["Dam Name", "Type", "Inundation Area (sq km)"]
    print("\nTop", limit, "Utah dams by inundation area:")
    print(tabulate([[r[0], r[1], round(r[2], 2)] for r in rows], headers, tablefmt="grid"))
    return rows  # [(name, type, area)]

In [5]:
def analyze_power_plants_at_risk(conn, dam_name):
    """Print and return the power plants whose points fall within *dam_name* polygon."""
    with conn.cursor() as cur:
        # Column discovery – dams
        dam_cols = get_actual_column_names(conn, "utah_dams")
        name_col_dam = next(c for c in dam_cols if c.upper() == "NAME")
        geom_col_dam = "geom"

        # Column discovery – power plants
        pp_cols = get_actual_column_names(conn, "power_plants")
        name_col_pp = next(c for c in pp_cols if c.upper() == "NAME")
        type_col_pp = next(c for c in pp_cols if c.upper() == "TYPE")
        fuel_col = next(c for c in pp_cols if c.upper() == "PRIM_FUEL")
        cap_col = next(c for c in pp_cols if c.upper() == "SUMMER_CAP")
        geom_col_pp = "geom"

        # Spatial query – intersects rather than contains (safer for points)
        cur.execute(
            f"""
            SELECT p."{name_col_pp}", p."{type_col_pp}", p."{fuel_col}", p."{cap_col}"
            FROM   utah_dams d
            JOIN   power_plants p
              ON   ST_Intersects(d."{geom_col_dam}", p."{geom_col_pp}")
            WHERE  d."{name_col_dam}" = %s
            ORDER  BY p."{cap_col}" DESC NULLS LAST
            """,
            (dam_name,),
        )
        rows = cur.fetchall()

    print("\n--- Power Plants at Risk ---")
    if rows:
        headers = ["Plant Name", "Type", "Primary Fuel", "Capacity (MW)"]
        print(tabulate(rows, headers, tablefmt="grid"))
    else:
        print(f"No power plants found within the inundation zone of '{dam_name}'.")
    return rows

In [6]:
dams = list_available_dams(conn)
print("\nEnter a dam name to analyze or type 'ALL' for every dam")


Top 50 Utah dams by inundation area:
+--------------------------------------+-----------+---------------------------+
| Dam Name                             | Type      |   Inundation Area (sq km) |
+======================================+===========+===========================+
| SEVIER BRIDGE                        | PMF       |                    863.32 |
+--------------------------------------+-----------+---------------------------+
| SEVIER BRIDGE                        | Sunny     |                    808.46 |
+--------------------------------------+-----------+---------------------------+
| DMAD                                 | PMF       |                    581.88 |
+--------------------------------------+-----------+---------------------------+
| PIUTE                                | PMF       |                    224.4  |
+--------------------------------------+-----------+---------------------------+
| SETTLEMENT CANYON                    | PMF       |                   

In [11]:
selected = input("Dam name: ").strip()
if not selected:
    selected = dams[0][0]  # largest dam by area

if selected.upper() == "ALL":
    dam_names = [d[0] for d in dams]
else:
    dam_names = [selected]

Dam name:  ALL


In [12]:
for dam in dam_names:
    print("\n" + "=" * 60)
    print(f"POWER-PLANT RISK REPORT FOR {dam}")
    print("=" * 60)
    plants = analyze_power_plants_at_risk(conn, dam)
    print(f"\nTotal plants at risk: {len(plants)}\n")


POWER-PLANT RISK REPORT FOR SEVIER BRIDGE

--- Power Plants at Risk ---
No power plants found within the inundation zone of 'SEVIER BRIDGE'.

Total plants at risk: 0


POWER-PLANT RISK REPORT FOR SEVIER BRIDGE

--- Power Plants at Risk ---
No power plants found within the inundation zone of 'SEVIER BRIDGE'.

Total plants at risk: 0


POWER-PLANT RISK REPORT FOR DMAD

--- Power Plants at Risk ---
No power plants found within the inundation zone of 'DMAD'.

Total plants at risk: 0


POWER-PLANT RISK REPORT FOR PIUTE

--- Power Plants at Risk ---
No power plants found within the inundation zone of 'PIUTE'.

Total plants at risk: 0


POWER-PLANT RISK REPORT FOR SETTLEMENT CANYON

--- Power Plants at Risk ---
No power plants found within the inundation zone of 'SETTLEMENT CANYON'.

Total plants at risk: 0


POWER-PLANT RISK REPORT FOR NEWCASTLE

--- Power Plants at Risk ---
No power plants found within the inundation zone of 'NEWCASTLE'.

Total plants at risk: 0


POWER-PLANT RISK REPORT F

In [9]:
conn.close()